In [19]:
import os

os.makedirs("./data", exist_ok=True)

with open("./data/cafe_menu.txt", "w", encoding="utf-8") as f:
    f.write("""\
메뉴번호: 1
이름: 아메리카노
가격: ₩4,500
재료: 에스프레소, 뜨거운 물
설명: 원두 본연의 맛을 즐길 수 있는 기본 커피

메뉴번호: 2
이름: 카페라떼
가격: ₩5,000
재료: 에스프레소, 우유
설명: 부드럽고 고소한 맛의 밀크커피

메뉴번호: 3
이름: 바닐라라떼
가격: ₩5,500
재료: 에스프레소, 우유, 바닐라 시럽
설명: 달콤한 바닐라 향이 가미된 라떼

메뉴번호: 4
이름: 콜드브루
가격: ₩5,000
재료: 콜드브루 원액, 물
설명: 깔끔하고 부드러운 맛의 차가운 커피

메뉴번호: 5
이름: 헤이즐넛라떼
가격: ₩5,500
재료: 에스프레소, 우유, 헤이즐넛 시럽
설명: 고소하고 달콤한 맛의 라떼

메뉴번호: 6
이름: 카푸치노
가격: ₩5,000
재료: 에스프레소, 우유 거품
설명: 풍부한 거품과 진한 커피의 조화

메뉴번호: 7
이름: 말차라떼
가격: ₩5,800
재료: 말차, 우유
설명: 쌉싸름한 말차와 우유의 조화

메뉴번호: 8
이름: 토피넛라떼
가격: ₩5,800
재료: 에스프레소, 우유, 토피넛 시럽
설명: 달콤하고 고소한 견과류 풍미의 라떼

메뉴번호: 9
이름: 민트초코라떼
가격: ₩6,000
재료: 우유, 민트 시럽, 초코 시럽
설명: 상쾌한 민트와 진한 초코의 만남

메뉴번호: 10
이름: 딸기라떼
가격: ₩5,500
재료: 우유, 딸기청
설명: 상큼한 딸기의 향과 부드러운 우유
""")


### 문제 5-1 : 카페 메뉴 도구(Tool) 호출 체인 구현

#### 1. 카페 메뉴 데이터 파일 생성 및 벡터 DB 구축

In [20]:
import os
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

# 1. cafe_menu.txt 생성 (10개 메뉴 항목)
os.makedirs("./data", exist_ok=True)
with open("./data/cafe_menu.txt", "w", encoding="utf-8") as f:
    f.write("""\
메뉴번호: 1
이름: 아메리카노
가격: ₩4,500
재료: 에스프레소, 뜨거운 물
설명: 원두 본연의 맛을 즐길 수 있는 기본 커피

메뉴번호: 2
이름: 카페라떼
가격: ₩5,000
재료: 에스프레소, 우유
설명: 부드럽고 고소한 맛의 밀크커피

메뉴번호: 3
이름: 바닐라라떼
가격: ₩5,500
재료: 에스프레소, 우유, 바닐라 시럽
설명: 달콤한 바닐라 향이 가미된 라떼

메뉴번호: 4
이름: 콜드브루
가격: ₩5,000
재료: 콜드브루 원액, 물
설명: 깔끔하고 부드러운 맛의 차가운 커피

메뉴번호: 5
이름: 헤이즐넛라떼
가격: ₩5,500
재료: 에스프레소, 우유, 헤이즐넛 시럽
설명: 고소하고 달콤한 맛의 라떼

메뉴번호: 6
이름: 카푸치노
가격: ₩5,000
재료: 에스프레소, 우유 거품
설명: 풍부한 거품과 진한 커피의 조화

메뉴번호: 7
이름: 말차라떼
가격: ₩5,800
재료: 말차, 우유
설명: 쌉싸름한 말차와 우유의 조화

메뉴번호: 8
이름: 토피넛라떼
가격: ₩5,800
재료: 에스프레소, 우유, 토피넛 시럽
설명: 달콤하고 고소한 견과류 풍미의 라떼

메뉴번호: 9
이름: 민트초코라떼
가격: ₩6,000
재료: 우유, 민트 시럽, 초코 시럽
설명: 상쾌한 민트와 진한 초코의 만남

메뉴번호: 10
이름: 딸기라떼
가격: ₩5,500
재료: 우유, 딸기청
설명: 상큼한 딸기의 향과 부드러운 우유
""")

# 2. 텍스트 → Document 로드 및 분할
loader = TextLoader("./data/cafe_menu.txt", encoding="utf-8")
raw_docs = loader.load()

splitter = CharacterTextSplitter(separator="\n\n", chunk_size=300, chunk_overlap=0)
docs = splitter.split_documents(raw_docs)

# 3. 벡터화 및 DB 저장
embedding_model = OllamaEmbeddings(model="bge-m3")
vectorstore = FAISS.from_documents(docs, embedding_model)
vectorstore.save_local("./db/cafe_db")


#### 2. 3개의 도구를 정의하고 LLM에 바인딩

In [21]:
from langchain_core.tools import tool
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.document_loaders import WikipediaLoader
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import ChatOpenAI

# a) tavily_search_func - 웹 검색 도구
tavily_search_func = TavilySearchResults(k=3)

# b) wiki_summary - 위키백과 요약 도구
@tool
def wiki_summary(query: str) -> str:
    """위키피디아에서 일반 지식을 검색하고 요약합니다."""
    docs = WikipediaLoader(query=query, lang="ko").load()
    return "\n".join([d.page_content for d in docs])

# c) db_search_cafe_func - 로컬 벡터 DB 검색 도구
@tool
def db_search_cafe_func(query: str) -> str:
    """카페 메뉴 정보를 벡터 DB에서 검색합니다."""
    vectorstore = FAISS.load_local("./db/cafe_db", OllamaEmbeddings(model="bge-m3"))
    retriever = vectorstore.as_retriever(search_type="similarity", k=3)
    results = retriever.get_relevant_documents(query)
    if not results:
        return "관련 정보를 찾을 수 없습니다."
    return "\n".join([f"- {doc.page_content}" for doc in results])

# ✅ LLM 바인딩
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
llm_with_tools = llm.bind_tools([
    tavily_search_func,
    wiki_summary,
    db_search_cafe_func
])


#### 3. 간단한 도구 호출 체인 구현체인 구조:

In [25]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import chain, RunnableConfig

# 1. 프롬프트 정의
prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 카페 정보에 대한 질문을 도와주는 AI 어시스턴트입니다. 다음은 도구 검색 결과입니다:\n{tool_messages}"),
    ("human", "{input}")
])

# 2. 도구 호출 및 결과 종합 체인
@chain
def cafe_tool_chain(user_input: str, config: RunnableConfig):
    # (1) 사용자 질문 → 도구 선택
    ai_msg = (prompt | llm_with_tools).invoke({"input": user_input}, config=config)

    # ✅ 도구 호출 리스트 확인
    print("🔍 선택된 도구 목록:", ai_msg.tool_calls)

    # (2) 도구 실행
    tool_messages = []
    for tool_call in ai_msg.tool_calls:
        tool_name = tool_call["name"]
        tool_args = tool_call["args"]

        if tool_name == "db_search_cafe_func":
            tool_messages.append(db_search_cafe_func.invoke(tool_args))
        elif tool_name == "wiki_summary":
            tool_messages.append(wiki_summary.invoke(tool_args))
        elif tool_name == "tavily_search":
            tool_messages.append(tavily_search_func.invoke(tool_args))

    # ✅ 도구 결과 확인
    print("🧾 도구 실행 결과:", tool_messages)

    # (3) 최종 답변 생성
    tool_messages_text = "\n\n".join(tool_messages)
    final_response = (prompt | llm).invoke({
        "input": user_input,
        "tool_messages": tool_messages_text
    })

    return final_response.content



#### 4. 테스트 질문 처리

In [14]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import chain, RunnableConfig

prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 카페 정보에 대한 질문을 도와주는 AI 어시스턴트입니다."),
    ("human", "{input}")
])

@chain
def cafe_tool_chain(user_input: str, config: RunnableConfig):
    ai_msg = (prompt | llm_with_tools).invoke({"input": user_input}, config=config)

    tool_messages = []
    for tool_call in ai_msg.tool_calls:
        tool_name = tool_call["name"]
        args = tool_call["args"]

        if tool_name == "db_search_cafe_func":
            tool_messages.append(db_search_cafe_func.invoke(args))
        elif tool_name == "wiki_summary":
            tool_messages.append(wiki_summary.invoke(args))
        elif tool_name == "tavily_search":
            tool_messages.append(tavily_search_func.invoke(args))

    final_response = (prompt | llm).invoke({
        "input": user_input,
        "tool_messages": tool_messages
    })

    return final_response.content

